In [ ]:
<a class="file" href="https://nsearchives.nseindia.com/content/indices/ind_nifty50list.csv" target="_blank">Download List of Nifty 50 stocks (.csv)</a>

In [1]:
import requests
import pandas as pd
import os
import time
import yfinance as yf

# Create directories for saving files
os.makedirs("nifty50_short_term_data", exist_ok=True)

# Step 1: Download Latest NIFTY 50 Stock List
def download_nifty50_list():
    url = "https://nsearchives.nseindia.com/content/indices/ind_nifty50list.csv"
    file_path = "nifty50_stocks.csv"

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; rv:102.0) Gecko/20100101 Firefox/102.0"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        with open(file_path, "wb") as file:
            file.write(response.content)
        print(f"✅ NIFTY 50 stock list downloaded successfully: {file_path}")

        df = pd.read_csv(file_path)
        df.columns = df.columns.str.strip()  # Clean column names
        return df
    else:
        print("❌ Failed to download NIFTY 50 stock list.")
        return None

# Step 2: Download Short-Term Price Data using yfinance
def download_stock_data_yfinance(stock_symbol, session):
    try:
        stock = yf.Ticker(stock_symbol, session=session)
        data = stock.history(period="1y", interval="1d")  # Fetch daily data for last 1 year

        if not data.empty:
            data.to_csv(f"nifty50_short_term_data/{stock_symbol}_1yr_data.csv")
            print(f"✅ Downloaded 1-year daily data for {stock_symbol}")
        else:
            print(f"⚠️ No data available for {stock_symbol}")

    except Exception as e:
        print(f"❌ Error fetching data for {stock_symbol}: {e}")

# Step 3: Handle IP Blocking by Using a Custom Session
def get_custom_yfinance_session():
    session = requests.Session()
    session.headers.update({"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})
    return session

# Main Execution Flow
nifty50_df = download_nifty50_list()
if nifty50_df is not None:
    # Save stock list to CSV
    nifty50_df.to_csv("nifty50_stocks.csv", index=False)
    print("📁 Saved NIFTY 50 stock list to nifty50_stocks.csv.")

    # Get a session with a custom user-agent to avoid blocking
    session = get_custom_yfinance_session()

    # Download short-term price data for each stock
    for symbol in nifty50_df["Symbol"]:
        full_symbol = f"{symbol}.NS"  # NSE India format for yfinance
        download_stock_data_yfinance(full_symbol, session)

        # Avoid getting blocked by adding delay
        time.sleep(2)

print("\n✅ All short-term NIFTY 50 stock data downloaded successfully!")


✅ NIFTY 50 stock list downloaded successfully: nifty50_stocks.csv
📁 Saved NIFTY 50 stock list to nifty50_stocks.csv.
✅ Downloaded 1-year daily data for ADANIENT.NS
✅ Downloaded 1-year daily data for ADANIPORTS.NS
✅ Downloaded 1-year daily data for APOLLOHOSP.NS
✅ Downloaded 1-year daily data for ASIANPAINT.NS
✅ Downloaded 1-year daily data for AXISBANK.NS
✅ Downloaded 1-year daily data for BAJAJ-AUTO.NS
✅ Downloaded 1-year daily data for BAJFINANCE.NS
✅ Downloaded 1-year daily data for BAJAJFINSV.NS
✅ Downloaded 1-year daily data for BEL.NS
✅ Downloaded 1-year daily data for BPCL.NS
✅ Downloaded 1-year daily data for BHARTIARTL.NS
✅ Downloaded 1-year daily data for BRITANNIA.NS
✅ Downloaded 1-year daily data for CIPLA.NS
✅ Downloaded 1-year daily data for COALINDIA.NS
✅ Downloaded 1-year daily data for DRREDDY.NS
✅ Downloaded 1-year daily data for EICHERMOT.NS
✅ Downloaded 1-year daily data for GRASIM.NS
✅ Downloaded 1-year daily data for HCLTECH.NS
✅ Downloaded 1-year daily data for H

In [1]:
import os
import datetime
import time
import pandas as pd
import requests
from openchart import NSEData

# Initialize OpenChart NSEData
nse = NSEData()

# Retry downloading master data if it fails
max_retries = 5
retry_count = 0

while retry_count < max_retries:
    try:
        print(f"⏳ Attempt {retry_count + 1}: Downloading NSE Master Data...")
        nse.download()  # Removed the 'timeout' argument

        if 'NSE' in nse.symbols and not nse.symbols['NSE'].empty:
            print("✅ NSE master data downloaded successfully.")
            break
        else:
            print("⚠️ No data received. Retrying...")

    except Exception as e:
        print(f"❌ Error downloading NSE master data: {e}")

    retry_count += 1
    time.sleep(5)

if retry_count == max_retries:
    print("❌ Failed to download NSE master data after multiple attempts.")
    exit()

# Load NIFTY 50 Stock List
nifty50_csv_url = "https://archives.nseindia.com/content/indices/ind_nifty50list.csv"
nifty50_csv_file = "nifty50_stocks.csv"

try:
    response = requests.get(nifty50_csv_url, timeout=10)
    response.raise_for_status()  # Raise an error for bad status codes

    with open(nifty50_csv_file, "wb") as file:
        file.write(response.content)
    print(f"✅ NIFTY 50 stock list downloaded: {nifty50_csv_file}")

except requests.exceptions.RequestException as e:
    print(f"❌ Failed to download NIFTY 50 stock list: {e}")
    exit()

# Read the CSV into a DataFrame
nifty50_df = pd.read_csv(nifty50_csv_file)

# Define Timeframes and Lookback Periods
timeframes = {
    '1m': 2,
    '3m': 2,
    '5m': 2,
    '10m': 2,
    '15m': 2,
    '30m': 2,
    '1h': 3
}

# Create Directory for Data Storage
os.makedirs("nifty50_intraday_data", exist_ok=True)

# Fetch and Save Intraday Data
def download_intraday_data(symbol):
    """Fetch and save intraday data for different timeframes."""
    for interval, days in timeframes.items():
        try:
            print(f"📥 Downloading {interval} data for {symbol} ({days} days)...")
            end_date = datetime.datetime.now()
            start_date = end_date - datetime.timedelta(days=days)

            df = nse.historical(
                symbol=symbol,
                exchange='NSE',
                start=start_date,
                end=end_date,
                interval=interval
            )

            if df is not None and not df.empty:
                df.reset_index(inplace=True)  # Ensure Timestamp is a column
                file_path = f"nifty50_intraday_data/{symbol}_{interval}.csv"
                df.to_csv(file_path, index=False)  # Save without extra index
                print(f"✅ Saved {interval} data to {file_path}")
            else:
                print(f"⚠️ No data available for {symbol} ({interval})")

            time.sleep(1)  # Avoid API rate limits

        except Exception as e:
            print(f"❌ Error fetching {interval} data for {symbol}: {e}")
            with open("error_log.txt", "a") as log_file:
                log_file.write(f"Error fetching {interval} data for {symbol}: {e}\n")

# Loop Through NIFTY 50 Stocks
for symbol in nifty50_df["Symbol"]:
    download_intraday_data(symbol)

print("\n✅ All intraday data downloaded successfully!")


⏳ Attempt 1: Downloading NSE Master Data...
NSE data shape: (2917, 4)
NFO data shape: (79961, 4)
NSE and NFO data downloaded successfully.
❌ Error downloading NSE master data: 'NSEData' object has no attribute 'symbols'
⏳ Attempt 2: Downloading NSE Master Data...
NSE data shape: (2917, 4)
NFO data shape: (79961, 4)
NSE and NFO data downloaded successfully.
❌ Error downloading NSE master data: 'NSEData' object has no attribute 'symbols'
⏳ Attempt 3: Downloading NSE Master Data...
NSE data shape: (2917, 4)
NFO data shape: (79961, 4)
NSE and NFO data downloaded successfully.
❌ Error downloading NSE master data: 'NSEData' object has no attribute 'symbols'
⏳ Attempt 4: Downloading NSE Master Data...
NSE data shape: (2917, 4)
NFO data shape: (79961, 4)
NSE and NFO data downloaded successfully.
❌ Error downloading NSE master data: 'NSEData' object has no attribute 'symbols'
⏳ Attempt 5: Downloading NSE Master Data...
NSE data shape: (2917, 4)
NFO data shape: (79961, 4)
NSE and NFO data downlo